In [1]:
import math
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import ot as pot
import torch
import torchdyn
from torchdyn.core import NeuralODE
from torchdyn.datasets import generate_moons

from torchcfm.conditional_flow_matching import *
from torchcfm.models.models import *
from torchcfm.utils import *

savedir = "models/8gaussian-moons_sinkhorn"
os.makedirs(savedir, exist_ok=True)

In [2]:
blur = 0.05
scaling = 0.95
backend = 'online'
lr = 0.01
device = 'cuda:0'
t = 100
dim = 2
batch_size = 256
model = MLP(dim=dim, time_varying=True)
optimizer = torch.optim.Adam(model.parameters())
FM = SinkhornFlowMatcher(blur = blur, scaling = scaling, backend = backend, lr = lr, device = device)
start = time.time()
for k in range(20000):
    optimizer.zero_grad()

    x0 = sample_8gaussians(batch_size).to(device)
    x1 = sample_moons(batch_size).to(device)

    t_train, xt, ut = FM.sample_location_and_conditional_flow(x0, x1, t)
    FM.clear_all()

    vt = model(torch.cat([xt.to('cpu'), t_train[:, None].to('cpu')], dim=-1))
    loss = torch.mean((vt - ut.to('cpu')) ** 2)

    loss.backward()
    optimizer.step()

    if (k + 1) % 5000 == 0:
        end = time.time()
        print(f"{k+1}: loss {loss.item():0.3f} time {(end - start):0.2f}")
        start = end
        node = NeuralODE(
            torch_wrapper(model), solver="dopri5", sensitivity="adjoint", atol=1e-4, rtol=1e-4
        )
        with torch.no_grad():
            traj = node.trajectory(
                sample_8gaussians(1024),
                t_span=torch.linspace(0, 1, 100),
            )
            plot_trajectories(traj)
        break
torch.save(model, f"{savedir}/cfm_v1.pt")


[KeOps] error: cuMemAlloc(&p_data, sizeof(TYPE *) * nargs) failed with error CUDA_ERROR_OUT_OF_MEMORY



RuntimeError: [KeOps] Cuda error.